In [160]:
from bs4 import BeautifulSoup
import requests
import math
from dateutil import parser
import html
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.action_chains import ActionChains


In [161]:
def scrapCorpus(sources,keyword,nb_docs):

    source = ""
    position = ""
    company = ""
    workplace = ""
    published_date = ""
    contract_type = ""
    long_infos = ""
    # 20 offres par pages
    pages = math.ceil(nb_docs/20)

    if  "emploi-territorial" in sources:
        source = "emploi-territorial"
        rootLink = "https://www.emploi-territorial.fr"
        # 20 offres par pages
        # pages = math.ceil(nb_docs/20)
        url = f"{rootLink}/emploi-mobilite/?adv-search={keyword}&page={pages}"
        response = requests.get(url) 
        soup = BeautifulSoup(response.content, 'html.parser')
        root = soup.find("body")
        offresLinkElems = root.select("div[class*='bloc-lien-offre'] > a[class*='lien-details-offre']")[:nb_docs]    
        links = [rootLink+offresLinkElem.get("href") for offresLinkElem in offresLinkElems]

        corpus = list(dict())

        for link in links:
            response = requests.get(link)
            soup = BeautifulSoup(response.content, 'html.parser')
            
            root = soup.find("body")

            position = root.select("h2[class*='set-line-emploi']")[0].text.strip()
            company = root.select("div[class*='offre-item-value'] > strong > a")[0].text.strip() if root.select("div[class*='offre-item-value'] > strong > a") else "NULL"
            workplace = root.select_one("div[class*='offre-item-label']:contains('Lieu de travail') + .offre-item-value").text.strip() if root.select_one("div[class*='offre-item-label']:contains('Lieu de travail') + .offre-item-value") else "NULL"
            published_date = root.select_one("div[class*='px-3']:contains('Publiée le') > .set-color-green").text.strip() if root.select_one("div[class*='px-3']:contains('Publiée le') > .set-color-green") else "NULL"
            contract_type = root.select_one('div[class*="offre-item-label"]:contains("Type d\'emploi") + .offre-item-value').text.strip() if root.select_one('div[class*="offre-item-label"]:contains("Type d\'emploi") + .offre-item-value') else "NULL"
            position_type = root.select_one('div[class*="offre-item-label"]:contains("Famille de métiers") + .offre-item-value').text.split(">")[0].strip()
            
            long_text = root.select('div[class*="offre-item-text"]')

            long_infos = " ".join([long_text[0].text.strip(),long_text[1].text.strip(),long_text[2].text.strip()])

            corpus.append({"source":source,"link":rootLink,"position":position,"position_type":position_type,"company":company,"workplace":workplace,"published_date":published_date,"contract_type":contract_type,"description":long_infos})
    
    if "apec" in sources:
        source = "apec"
        service = ChromeService("C:/Users/leogo/Documents/chromedriver-win64/chromedriver-win64/chromedriver.exe")
        driver = webdriver.Chrome(service=service)
        rootLink = "https://www.apec.fr"
        try:
            for page in range(pages):    
                try:
                    driver.get(f'{rootLink}/candidat/recherche-emploi.html/emploi?motsCles={keyword}&page={pages}')
                    # Initial find of elements
                    try:
                        deny_cookies_button = WebDriverWait(driver, 10).until(
                            EC.element_to_be_clickable((By.CSS_SELECTOR, 'button#onetrust-reject-all-handler'))
                        )

                        deny_cookies_button.click()
                    except TimeoutException:
                        pass

                    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                    WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.CSS_SELECTOR, "div.card-offer"))
                    )

                    # Getting docs left if last page to query
                    if nb_docs%20 != 0 and page==pages-1: 
                        limit = nb_docs%20+1
                        div_elements_to_click_list = driver.find_elements(By.CSS_SELECTOR, 'div.card-offer')[:limit]
                    else :
                        div_elements_to_click_list = driver.find_elements(By.CSS_SELECTOR, 'div.card-offer')
                        
                    for index in range(len(div_elements_to_click_list)):
                        try:
                            WebDriverWait(driver, 10).until(
                                EC.presence_of_element_located((By.CSS_SELECTOR, "div.card-offer"))
                            )
                            # Re-find elements after navigating back
                            div_elements_to_click_list = driver.find_elements(By.CSS_SELECTOR, 'div.card-offer')

                            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                            WebDriverWait(driver, 10).until(
                                EC.presence_of_element_located((By.CSS_SELECTOR, "div.card-offer"))
                            )
                            # Check if the index is within the valid range

                            div_elements_to_click = div_elements_to_click_list[index]
                            company = div_elements_to_click.find_element(By.CSS_SELECTOR, 'p.card-offer__company').text
                            contract_type = div_elements_to_click.find_elements(By.CSS_SELECTOR, 'ul.important-list > li')[0].text
                            workplace = div_elements_to_click.find_elements(By.CSS_SELECTOR, 'ul.important-list > li')[1].text
                            published_date = div_elements_to_click.find_elements(By.CSS_SELECTOR, 'ul.important-list > li')[2].text
                            # Scroll into view

                            actions = ActionChains(driver)
                            actions.move_to_element(div_elements_to_click).click().perform()

                            # Wait for the child element to become present in the DOM
                            WebDriverWait(driver, 20).until(
                                EC.presence_of_element_located((By.XPATH, f"//h4[text()='Descriptif du poste']"))
                            )
                            # Get the page source after the click
                            page_source = driver.page_source

                            # Use Beautiful Soup to parse the page source
                            soup = BeautifulSoup(page_source, 'html.parser')

                            # Example: Retrieve the text of a specific element
                            desc = soup.select("h4:contains('Descriptif du poste') + p")
                            descText = ["".join(elem.text) for elem in desc][0]

                            profile = soup.select("h4:contains('Profil recherché') + p")
                            profileText = ["".join(elem.text) for elem in profile][0]                  

                            position = soup.select_one("h4:contains('Métier') + span").text
                            position_type = soup.select_one('h4:contains("Secteur d’activité du poste")+span').text
                            long_infos = " ".join([descText,profileText])
                            
                            corpus.append({"source":source,"link":rootLink,"position":position,"position_type":position_type,"company":company,"workplace":workplace,"published_date":published_date,"contract_type":contract_type,"description":long_infos})
                        except Exception as e:
                            # Print the exception for debugging purposes
                            print(f"Error: {e}")

                        finally:
                            # Navigate back to the main page
                            driver.execute_script("window.history.go(-1);")

                except Exception as e:
                    # Print the exception for debugging purposes
                    print(f"Error: {e}")

        finally:
            driver.quit()
    return(corpus)


In [162]:
corpus = scrapCorpus(sources=["emploi-territorial","apec"],keyword="data",nb_docs=22)

Error: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF783EF82B2+55298]
	(No symbol) [0x00007FF783E65E02]
	(No symbol) [0x00007FF783D205AB]
	(No symbol) [0x00007FF783D6175C]
	(No symbol) [0x00007FF783D618DC]
	(No symbol) [0x00007FF783D9CBC7]
	(No symbol) [0x00007FF783D820EF]
	(No symbol) [0x00007FF783D9AAA4]
	(No symbol) [0x00007FF783D81E83]
	(No symbol) [0x00007FF783D5670A]
	(No symbol) [0x00007FF783D57964]
	GetHandleVerifier [0x00007FF784270AAB+3694587]
	GetHandleVerifier [0x00007FF7842C728E+4048862]
	GetHandleVerifier [0x00007FF7842BF173+4015811]
	GetHandleVerifier [0x00007FF783F947D6+695590]
	(No symbol) [0x00007FF783E70CE8]
	(No symbol) [0x00007FF783E6CF34]
	(No symbol) [0x00007FF783E6D062]
	(No symbol) [0x00007FF783E5D3A3]
	BaseThreadInitThunk [0x00007FFAAFEA257D+29]
	RtlUserThreadStart [0x00007FFAB19EAA58+40]



In [163]:
from sqlalchemy import create_engine, Column, Integer, Date, ForeignKey,text
from sqlalchemy.orm import declarative_base, Session, relationship
from sqlalchemy.orm import sessionmaker

Base = declarative_base()

# Modele de bdd
class DPosition(Base):
    __tablename__ = 'd_position'
    id = Column(Integer, primary_key=True)
    position = Column(String)

class DWebsite(Base):
    __tablename__ = 'd_website'
    id = Column(Integer, primary_key=True)
    label = Column(String)
    link = Column(String)

class DCompany(Base):
    __tablename__ = 'd_company'
    id = Column(Integer, primary_key=True)
    label = Column(String)

class DCity(Base):
    __tablename__ = 'd_city'
    id = Column(Integer, primary_key=True)
    city = Column(String)

class DContractType(Base):
    __tablename__ = 'd_contract_type'
    id = Column(Integer, primary_key=True)
    contract_type = Column(String)


class FJobAdvertisements(Base):
    __tablename__ = 'f_job_advertisements'
    nb_occurences = Column(Integer)
    # published_date = Column(Date, primary_key=True)
    contract_type_id = Column(Integer, ForeignKey('d_contract_type.id'),primary_key=True)
    position_id = Column(Integer, ForeignKey('d_position.id'),primary_key=True)
    website_id = Column(Integer, ForeignKey('d_website.id'),primary_key=True)
    city_id = Column(Integer, ForeignKey('d_city.id'),primary_key=True)
    company_id = Column(Integer, ForeignKey('d_company.id'),primary_key=True)
    
    position = relationship('DPosition', back_populates='job_advertisements')
    website = relationship('DWebsite', back_populates='job_advertisements')
    company = relationship('DCompany', back_populates='job_advertisements')
    city = relationship('DCity', back_populates='job_advertisements')
    contract_type = relationship('DContractType', back_populates='job_advertisements')

# Définition des relations
DPosition.job_advertisements = relationship('FJobAdvertisements', back_populates='position')
DWebsite.job_advertisements = relationship('FJobAdvertisements', back_populates='website')
DCompany.job_advertisements = relationship('FJobAdvertisements', back_populates='company')
DCity.job_advertisements = relationship('FJobAdvertisements', back_populates='city')
DContractType.job_advertisements = relationship('FJobAdvertisements', back_populates='contract_type')


In [167]:
#Alimentation du DW
def fillDW(corpus):
    engine = create_engine('mysql+mysqlconnector://root:@localhost/job_scrapping')
    Base.metadata.create_all(engine)
    Session = sessionmaker(bind=engine)

    #Vidage du DW
    session = Session()
    session.execute(text("CALL pTruncateDW()"))
    session.commit()

    #Itération sur les éléments du corpus récupérés
    for i,item in enumerate(corpus):
        corpus_position = item["position"]
        corpus_website = item["source"]
        corpus_link = item["link"]
        corpus_company = item["company"]
        corpus_city = item["workplace"]
        corpus_contract_type = item["contract_type"]

        position = DPosition(position=corpus_position)
        website = DWebsite(label=corpus_website,link=corpus_link)
        company = DCompany(label=corpus_company)
        city = DCity(city=corpus_city)
        contract_type = DContractType(contract_type=corpus_contract_type)

        session = Session()

        # Recherche des élements de chaque dimensions pour éviter les doublons
        existing_position = session.query(DPosition).filter_by(position=corpus_position).first()
        existing_website = session.query(DWebsite).filter_by(label=corpus_website).first()
        existing_company = session.query(DCompany).filter_by(label=corpus_company).first()
        existing_city = session.query(DCity).filter_by(city=corpus_city).first()
        existing_contract_type = session.query(DContractType).filter_by(contract_type=corpus_contract_type).first()

        #Si elle existe je récupère la ligne existante, sinon j'insert la nouvelle ligne
        if existing_position:
            position = existing_position
        else:
            session.add(position)

        if existing_website:
            website = existing_website
        else:
            session.add(website)

        if existing_company:
            company = existing_company
        else:
            session.add(company)

        if existing_city:
            city = existing_city
        else:
            session.add(city)

        if existing_contract_type:
            contract_type = existing_contract_type
        else:
            session.add(contract_type)
        session.commit()

        # J'insert les données dans la table de fait (id des dimensions + KPI)
        job_advertisement = FJobAdvertisements(
            nb_occurences=10,
            position=position,
            website=website,
            company=company,
            city=city,
            contract_type=contract_type
        )

        session.add(job_advertisement)
        session.commit()
        print(i)
    session.close()

In [168]:
fillDW(corpus)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41


IntegrityError: (mysql.connector.errors.IntegrityError) 1062 (23000): Duplicate entry '1404-944-224-189-184' for key 'PRIMARY'
[SQL: INSERT INTO f_job_advertisements (nb_occurences, contract_type_id, position_id, website_id, city_id, company_id) VALUES (%(nb_occurences)s, %(contract_type_id)s, %(position_id)s, %(website_id)s, %(city_id)s, %(company_id)s)]
[parameters: {'nb_occurences': 10, 'contract_type_id': 224, 'position_id': 189, 'website_id': 184, 'city_id': 1404, 'company_id': 944}]
(Background on this error at: https://sqlalche.me/e/20/gkpj)